In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import tkinter as tk
from tkinter import *
import json
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\slope\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\slope\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df_Productos = pd.read_csv('productos.csv')
df_Productos = df_Productos.drop(columns=['categoria1', 'categoria2', 'categoria3'])

df_Productos

,id,marca,nombre,presentacion
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg
...,...,...,...,...
72033,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Si...,500.0 gr
72034,9795403001143,MAYO,Mini Pizzetas Mayo 12 Un,12.0 un
72035,9990385651922,DANA,Te Negro en Hebras Lata Dana 50 Gr,50.0 gr
72036,9990385651939,DANA,Te Verde en Hebras Lata Dana 50 Gr,50.0 gr


In [3]:
df_Precios = pd.read_csv('precios_20200518_20200518.csv')
df_Precios = df_Precios.dropna(subset=['precio'])
df_Precios

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-009
1,32.90,0000000002288,2-1-037
2,36.90,0000000002288,2-1-090
3,39.90,0000000002288,2-3-247
4,499.99,0000000205870,9-1-430
...,...,...,...
415096,149.99,9569753142128,25-1-1
415097,34.99,9795403001143,25-1-1
415098,312.50,9990385651922,5-1-4
415099,312.50,9990385651939,5-1-4


In [4]:
df_Productos['id'] = df_Productos['id'].astype(str)
df_Precios['producto_id'] = df_Precios['producto_id'].astype(str)

# Unir los DataFrames en base al 'producto_id'
df_combined = pd.merge(df_Precios, df_Productos, left_on='producto_id', right_on='id')
df_combined

,precio,producto_id,sucursal_id,id,marca,nombre,presentacion
0,29.90,0000000002288,2-1-009,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
1,32.90,0000000002288,2-1-037,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
2,36.90,0000000002288,2-1-090,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
3,39.90,0000000002288,2-3-247,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
4,499.99,0000000205870,9-1-430,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg
...,...,...,...,...,...,...,...
413140,149.99,9569753142128,25-1-1,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Si...,500.0 gr
413141,34.99,9795403001143,25-1-1,9795403001143,MAYO,Mini Pizzetas Mayo 12 Un,12.0 un
413142,312.50,9990385651922,5-1-4,9990385651922,DANA,Te Negro en Hebras Lata Dana 50 Gr,50.0 gr
413143,312.50,9990385651939,5-1-4,9990385651939,DANA,Te Verde en Hebras Lata Dana 50 Gr,50.0 gr


In [10]:
df_combined.to_csv("productos_prueba.csv", index=0)

In [5]:
product_names = df_combined['nombre'].unique()
print(product_names.size)
product_names

53973


array(['Perejil Atado La Anonima 1 Un', 'Ojo de Bife 1 Kg',
       'Milanesa de Peceto Novillito 1 Kg', ...,
       'Te Negro en Hebras Lata Dana 50 Gr',
       'Te Verde en Hebras Lata Dana 50 Gr',
       'Yerba Mate Aromatizada Lata Dana 150 Gr'], dtype=object)

In [6]:
lemmatizer = WordNetLemmatizer()

# Datos de entrenamiento
intents = {
    "intents": [
        {"tag": "saludo",
         "patterns": ["Hola", "¿Cómo estás?", "Buenos días", "Buenas tardes"],
         "responses": ["Hola, ¿Cómo puedo ayudarte?", "¡Hola! ¿En qué puedo ayudarte?", "Hola, ¿cómo estás?"]},
        {"tag": "despedida",
         "patterns": ["Adiós", "Nos vemos", "Hasta luego", "Bye"],
         "responses": ["Adiós, que tengas un buen día", "Hasta luego, cuídate", "Nos vemos, ¡adiós!"]},
        {"tag": "gracias",
         "patterns": ["Gracias", "Muchas gracias", "Te lo agradezco"],
         "responses": ["De nada", "¡No hay problema!", "A la orden"]},
        {"tag": "nombre",
         "patterns": ["¿Cómo te llamas?", "¿Cuál es tu nombre?", "Dime tu nombre"],
         "responses": ["Soy un chatbot, tu asistente virtual", "Me llamo Chatbot", "Soy Chatbot, encantado de ayudarte"]},
        {"tag": "edad",
         "patterns": ["¿Cuántos años tienes?", "¿Qué edad tienes?", "Dime tu edad"],
         "responses": ["Soy tan viejo como el internet", "No tengo edad, soy un programa", "Mi edad es irrelevante, soy un chatbot"]},
        {"tag": "ubicacion",
         "patterns": ["¿Dónde estás?", "¿Dónde te encuentras?", "Dime tu ubicación"],
         "responses": ["Estoy en tu dispositivo", "Estoy en la nube", "Estoy donde tú estés, en tu dispositivo"]},
        {"tag": "trabajo",
         "patterns": ["¿Qué haces?", "¿Cuál es tu trabajo?", "¿En qué trabajas?"],
         "responses": ["Ayudo a las personas a responder sus preguntas", "Soy un asistente virtual", "Mi trabajo es asistirte"]},
        {"tag": "tiempo",
         "patterns": ["¿Qué hora es?", "Dime la hora", "¿Tienes la hora?"],
         "responses": ["No tengo un reloj, pero puedo decirte que es un buen momento para aprender algo nuevo", "Lo siento, no tengo la hora exacta", "El tiempo es relativo, disfruta el momento"]},
        {"tag": "clima",
         "patterns": ["¿Cómo está el clima?", "¿Qué clima hace?", "¿Cuál es el pronóstico del tiempo?"],
         "responses": ["No puedo comprobar el clima, pero espero que esté agradable", "No tengo acceso a datos meteorológicos, pero espero que haga buen tiempo", "No sé el clima exacto, pero siempre es un buen momento para una taza de café"]},
        {"tag": "dia_semana",
         "patterns": ["¿Qué día es hoy?", "Dime el día de hoy", "¿Qué día de la semana es?"],
         "responses": ["Hoy es un día maravilloso", "No tengo un calendario, pero cada día es una oportunidad", "El día exacto no lo sé, pero espero que sea un buen día para ti"]},
        {"tag": "juego",
         "patterns": ["¿Quieres jugar?", "Vamos a jugar", "¿Sabes algún juego?"],
         "responses": ["Podemos jugar a adivinar números", "¡Claro! Pensemos en un juego", "Podemos jugar a algo sencillo, dime qué quieres hacer"]},
        {"tag": "amor",
         "patterns": ["¿Qué es el amor?", "Háblame del amor", "Dime algo sobre el amor"],
         "responses": ["El amor es un sentimiento profundo de afecto", "El amor puede ser complejo y maravilloso", "El amor es algo que los humanos valoran mucho"]},
        {"tag": "comida",
         "patterns": ["¿Cuál es tu comida favorita?", "¿Te gusta la comida?", "Dime algo sobre la comida"],
         "responses": ["No como, pero sé que hay muchas comidas deliciosas", "Los humanos disfrutan de una gran variedad de alimentos", "La comida es esencial para los humanos, pero yo no la necesito"]},
        {"tag": "musica",
         "patterns": ["¿Te gusta la música?", "¿Qué tipo de música te gusta?", "Dime algo sobre la música"],
         "responses": ["La música es una forma maravillosa de expresión", "Hay muchos géneros de música, cada uno con su encanto", "La música puede cambiar el estado de ánimo y es muy poderosa"]},
        {"tag": "deporte",
         "patterns": ["¿Te gustan los deportes?", "¿Qué deporte te gusta?", "Dime algo sobre deportes"],
         "responses": ["Los deportes son una excelente manera de mantenerse activo", "Hay muchos deportes emocionantes, ¿cuál es tu favorito?", "Los deportes pueden ser muy competitivos y divertidos"]},
        {"tag": "pelicula",
         "patterns": ["¿Te gustan las películas?", "¿Qué película te gusta?", "Dime algo sobre películas"],
         "responses": ["Las películas son una gran forma de entretenimiento", "Hay muchos géneros de películas para disfrutar", "Las películas pueden contar historias increíbles"]},
        {"tag": "libro",
         "patterns": ["¿Te gustan los libros?", "¿Qué libro te gusta?", "Dime algo sobre libros"],
         "responses": ["Los libros son una puerta a mundos increíbles", "Hay libros para todos los gustos y edades", "La lectura es una excelente manera de aprender y divertirse"]},
        {"tag": "programacion",
         "patterns": ["¿Te gusta programar?", "¿Sabes programar?", "Dime algo sobre programación"],
         "responses": ["La programación es mi esencia", "Sí, sé programar. Estoy hecho de código", "La programación es una habilidad muy útil en el mundo moderno"]},
        {"tag": "viaje",
         "patterns": ["¿Te gusta viajar?", "¿A dónde te gustaría ir?", "Dime algo sobre viajes"],
         "responses": ["Viajar es una excelente manera de conocer nuevas culturas", "Hay muchos lugares increíbles para visitar en el mundo", "Viajar amplía tus horizontes y te enriquece como persona"]},
        {"tag": "chiste",
         "patterns": ["Cuéntame un chiste", "Dime un chiste", "¿Sabes algún chiste?"],
         "responses": ["¿Por qué la computadora fue al médico? Porque tenía un virus.", "¿Qué le dice una iguana a su hermana gemela? Somos iguanitas.", "¿Qué hace una abeja en el gimnasio? ¡Zum-ba!"]},
         {"tag": "greeting",
            "patterns": ["Hola", "Hey", "Hola, ¿cómo estás?", "Buenos días", "Buenas tardes"],
            "responses": ["Hola", "Hola, ¿cómo puedo ayudarte?", "Hola, ¿qué necesitas?"]},
        {"tag": "product_query",
            "patterns": ["¿Cuál es el precio de [producto]?", "Precio de [producto]", "¿Cuánto cuesta [producto]?"],
            "responses": ["Déjame buscar el precio para ti."]},
        {
            "tag": "products_list",
            "patterns": ["¿Qué productos tienes?", "Lista de productos", "¿Qué vendes?"],
            "responses": ["Estos son los productos disponibles: "]
        },

    ]
}

with open('intents.json', 'w') as outfile:
    json.dump(intents, outfile)

with open('intents.json') as file:
    data = json.load(file)

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']



# Procesamiento de datos
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)

train_x = []
train_y = []

for i in training:
    train_x.append(i[0])
    train_y.append(i[1])

train_x = np.array(train_x)
train_y = np.array(train_y)


In [7]:
# Función para limpiar y lematizar oración
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Función para convertir oración a bolsa de palabras
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return(np.array(bag))

# Función para predecir la clase
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print(return_list)
    return return_list

# Función para obtener respuesta
'''def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    print(list_of_intents)
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result'''

def get_response(ints, intents_json, msg):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    print(list_of_intents)


    if tag == 'products_list':
        response = "Estos son los productos disponibles:\n" + "\n".join(product_names)
        return response
    elif tag == 'product_query':
        product_name = msg.split('de')[-1].strip()
        prices = get_product_prices(product_name)
        if prices:
            response = f"Estos son los 10 precios más bajos que encontré para {product_name}:\n"
            sorted_prices = sorted(prices, key=lambda x: x['precio'])[:10]
            for price in sorted_prices:
                response += "- Sucursal {}: ${:.2f}\n".format(price['sucursal_id'], price['precio'])
            return response
        else:
            return "Lo siento, no encontré ese producto."
    else:
        for intent in data['intents']:
            if intent['tag'] == tag:
                responses = intent['responses']
        return random.choice(responses)    

def get_product_prices(product_name):
    result = df_combined[df_combined['nombre'].str.contains(product_name, case=False, na=False)]
    if not result.empty:
        return result[['sucursal_id', 'precio', 'marca', 'nombre', 'presentacion']].to_dict(orient='records')
    else:
        return None


In [8]:
class Chatbot:
    def __init__(self):
        self.model = self.create_model()
        self.train_model()
        self.intents = intents

    def create_model(self):
        model = Sequential()
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(len(train_y[0]), activation='softmax'))
        sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        return model

    def train_model(self):
        try:
            self.model.load_weights("model.weights.h5")
        except:
            self.model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
            self.model.save_weights("model.weights.h5")


    def chat(self, sentence):
        ints = predict_class(sentence, self.model)
        res = get_response(ints, self.intents, sentence)
        return res


chatbot = Chatbot()


c:\Users\slope\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\slope\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
c:\Users\slope\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
class ChatGUI:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Chatbot")
        self.chat_frame = Frame(self.window)
        self.chat_frame.pack(padx=10, pady=10)
        self.text_widget = Text(self.chat_frame, width=50, height=20)
        self.text_widget.pack(side=LEFT, padx=10, pady=10)
        self.text_widget.config(state=DISABLED)
        self.scrollbar = Scrollbar(self.chat_frame)
        self.scrollbar.pack(side=RIGHT, fill=Y)
        self.text_widget.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_widget.yview)
        self.entry = Entry(self.window, width=50)
        self.entry.pack(padx=10, pady=10)
        self.entry.bind("<Return>", self.send_message)
        self.send_button = Button(self.window, text="Enviar", command=self.send_message)
        self.send_button.pack(pady=10)

    def send_message(self, event=None):
        message = self.entry.get()
        self.entry.delete(0, END)
        if message:
            self.display_message(f"Tú: {message}\n")
            response = chatbot.chat(message)
            self.display_message(f"Bot: {response}\n")

    def display_message(self, message):
        self.text_widget.config(state=NORMAL)
        self.text_widget.insert(END, message)
        self.text_widget.config(state=DISABLED)
        self.text_widget.yview(END)

    def run(self):
        self.window.mainloop()

chat_gui = ChatGUI()
chat_gui.run()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[{'intent': 'products_list', 'probability': '0.99991906'}]
[{'tag': 'saludo', 'patterns': ['Hola', '¿Cómo estás?', 'Buenos días', 'Buenas tardes'], 'responses': ['Hola, ¿Cómo puedo ayudarte?', '¡Hola! ¿En qué puedo ayudarte?', 'Hola, ¿cómo estás?']}, {'tag': 'despedida', 'patterns': ['Adiós', 'Nos vemos', 'Hasta luego', 'Bye'], 'responses': ['Adiós, que tengas un buen día', 'Hasta luego, cuídate', 'Nos vemos, ¡adiós!']}, {'tag': 'gracias', 'patterns': ['Gracias', 'Muchas gracias', 'Te lo agradezco'], 'responses': ['De nada', '¡No hay problema!', 'A la orden']}, {'tag': 'nombre', 'patterns': ['¿Cómo te llamas?', '¿Cuál es tu nombre?', 'Dime tu nombre'], 'responses': ['Soy un chatbot, tu asistente virtual', 'Me llamo Chatbot', 'Soy Chatbot, encantado de ayudarte']}, {'tag': 'edad', 'patterns': ['¿Cuántos años tienes?', '¿Qué edad tienes?', 'Dime tu edad'], 'responses': ['Soy tan viejo como el internet', 'No tengo edad, soy un programa', 'Mi edad es i